# HAT Substrate Comparison: Neutral vs Emotional

This notebook compares **system-wide substrate signals** collected per run:
- `perf_stat.txt` (1 ms buckets from `sudo perf stat -I 1 ...`)
- `kernel_log.txt` (kernel log slice for the run window)
- `responses.jsonl` (request timing + success)

It intentionally **does not** treat `/proc` metrics as evidence, except for optional alignment/sanity.

You will set:
- `BASE_DIR`
- `NEUTRAL_RUN`
- `EMOTIONAL_RUN`


In [ ]:
import re, json
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams["figure.dpi"] = 120
plt.rcParams["figure.figsize"] = (12, 4)

## 1 — Configuration

In [ ]:
BASE_DIR = Path.home() / "Desktop" / "mccviahat" / "mccviahat_runs"

NEUTRAL_RUN   = "2026-02-09T22-28-54_neutral"
EMOTIONAL_RUN = "2026-02-09T22-18-50_emotional"

n_dir = BASE_DIR / NEUTRAL_RUN
e_dir = BASE_DIR / EMOTIONAL_RUN

for d in [n_dir, e_dir]:
    assert d.exists(), f"Missing: {d}"

print("Neutral:", n_dir.name)
print("Emotional:", e_dir.name)

## 2 — Loaders

In [ ]:
# ── Meta & responses ──

def load_json(path: Path) -> dict:
    return json.loads(path.read_text(encoding="utf-8"))

def load_responses(run_dir: Path) -> pd.DataFrame:
    rows = []
    for line in (run_dir / "responses.jsonl").read_text(encoding="utf-8").splitlines():
        if not line.strip():
            continue
        r = json.loads(line)
        # extract llama.cpp timings from the nested response JSON
        try:
            resp = json.loads(r["response_raw"])
            timings = resp.get("timings", {})
            r["prompt_ms"] = timings.get("prompt_ms")
            r["predicted_ms"] = timings.get("predicted_ms")
            r["tokens_evaluated"] = resp.get("tokens_evaluated")
            r["tokens_predicted"] = resp.get("tokens_predicted")
        except Exception:
            pass
        rows.append(r)
    df = pd.DataFrame(rows)
    keep = [c for c in ["id","title","ok","t_request_start_ns","t_request_end_ns",
                         "prompt_ms","predicted_ms","tokens_evaluated","tokens_predicted"] if c in df.columns]
    df = df[keep].sort_values("t_request_start_ns").reset_index(drop=True)
    df["duration_s"] = (df["t_request_end_ns"] - df["t_request_start_ns"]) / 1e9
    return df

In [ ]:
# ── perf stat parser ──
#
# Format per line (16 events x ~6000 buckets = ~97k lines):
#   0.001082375      1        irq:irq_handler_entry    #   1.536 /sec
#   <timestamp_s>  <count>    <event_name>             # <rate> <unit>
#
# Some counts contain commas (e.g. 88,790).  Some are floats (e.g. 651.08 msec cpu-clock).
# We extract: (timestamp_s, count_or_value, event_name).

def parse_perf_stat(path: Path) -> pd.DataFrame:
    """Parse `perf stat -I` output into a tidy DataFrame with columns: t_s, event, value."""
    text = path.read_text(encoding="utf-8", errors="replace")
    data = []
    # Match: <timestamp_s>  <value> [unit]  <event_name>
    # The '#' comment at the end is optional.
    pat = re.compile(
        r"^\s*"
        r"(\d+\.\d+)"          # group 1: timestamp in seconds
        r"\s+"
        r"([\d,\.]+|<[^>]+>)"  # group 2: count (possibly with commas) or <not counted>
        r"\s+"
        r"(?:\S+\s+)?"         # optional unit (msec, Joules, C, etc.)
        r"(\S+)"              # group 3: event name
        r"\s*",
        re.MULTILINE
    )
    for m in pat.finditer(text):
        t_s = float(m.group(1))
        raw = m.group(2).replace(",", "")
        evt = m.group(3)
        if raw.startswith("<"):
            val = np.nan
        else:
            try:
                val = float(raw)
            except ValueError:
                val = np.nan
        data.append((t_s, evt, val))
    return pd.DataFrame(data, columns=["t_s", "event", "value"])


def perf_to_wide(tidy: pd.DataFrame) -> pd.DataFrame:
    """Pivot tidy perf data to wide format: one row per timestamp, one column per event."""
    w = tidy.pivot_table(index="t_s", columns="event", values="value", aggfunc="first")
    w = w.sort_index().reset_index()
    return w

In [ ]:
# ── proc_system_sample.csv loader ──
#
# This CSV has a very wide header with /proc/interrupts IRQ numbers, softirq types,
# PSI fields, net/disk counters, and CPU freq.
# All values are cumulative counters (except PSI avgs and freq).

def load_proc_system(run_dir: Path) -> pd.DataFrame:
    df = pd.read_csv(run_dir / "proc_system_sample.csv")
    # Convert timestamp to relative seconds from first sample
    df["t_s"] = (df["timestamp_ns"] - df["timestamp_ns"].iloc[0]) / 1e9
    return df


def load_proc_sample(run_dir: Path) -> pd.DataFrame:
    df = pd.read_csv(run_dir / "proc_sample.csv")
    df["t_s"] = (df["timestamp_ns"] - df["timestamp_ns"].iloc[0]) / 1e9
    return df

## 3 — Load all data

In [ ]:
# Meta
n_meta  = load_json(n_dir / "meta.json")
e_meta  = load_json(e_dir / "meta.json")
n_cmeta = load_json(n_dir / "collector_meta.json")
e_cmeta = load_json(e_dir / "collector_meta.json")

# Responses
n_resp = load_responses(n_dir)
e_resp = load_responses(e_dir)

# Perf (1 ms resolution)
n_perf_tidy = parse_perf_stat(n_dir / "perf_stat.txt")
e_perf_tidy = parse_perf_stat(e_dir / "perf_stat.txt")
n_perf = perf_to_wide(n_perf_tidy)
e_perf = perf_to_wide(e_perf_tidy)

# /proc system (200 ms resolution)
n_sys = load_proc_system(n_dir)
e_sys = load_proc_system(e_dir)

# /proc process (200 ms resolution)
n_proc = load_proc_sample(n_dir)
e_proc = load_proc_sample(e_dir)

print(f"Perf events: {sorted(n_perf_tidy['event'].unique())}")
print(f"Perf buckets — neutral: {len(n_perf):,}  emotional: {len(e_perf):,}")
print(f"Proc system rows — neutral: {len(n_sys)}  emotional: {len(e_sys)}")
print(f"Requests — neutral: {len(n_resp)}  emotional: {len(e_resp)}")

## 4 — Request timing

In [ ]:
def request_windows(resp: pd.DataFrame, t0_ns: int) -> list[tuple[float, float]]:
    """Return list of (start_s, end_s) relative to collector t0."""
    wins = []
    for _, r in resp.iterrows():
        s = (r["t_request_start_ns"] - t0_ns) / 1e9
        e = (r["t_request_end_ns"]   - t0_ns) / 1e9
        wins.append((s, e))
    return wins

n_t0 = n_cmeta["t0_ns"]
e_t0 = e_cmeta["t0_ns"]
n_wins = request_windows(n_resp, n_t0)
e_wins = request_windows(e_resp, e_t0)

print("=== Neutral ===")
display(n_resp[["id", "title", "duration_s", "tokens_evaluated", "tokens_predicted", "prompt_ms", "predicted_ms"]])
print("\n=== Emotional ===")
display(e_resp[["id", "title", "duration_s", "tokens_evaluated", "tokens_predicted", "prompt_ms", "predicted_ms"]])

## 5 — Perf event exploration (1 ms resolution)

These are the high-resolution signals from `perf stat -I 1`. Each row is a ~1 ms bucket.

In [ ]:
def shade_requests(ax, wins, color="C0", alpha=0.08):
    for s, e in wins:
        ax.axvspan(s, e, color=color, alpha=alpha)

def plot_perf_event(event: str, smooth_ms: int = 50):
    """Plot a single perf event for both conditions, with request windows shaded."""
    if event not in n_perf.columns or event not in e_perf.columns:
        print(f"Event '{event}' not found in both runs.")
        return

    fig, axes = plt.subplots(1, 2, figsize=(14, 4), sharey=True)

    for ax, perf, wins, label, color in [
        (axes[0], n_perf, n_wins, "neutral", "C0"),
        (axes[1], e_perf, e_wins, "emotional", "C1"),
    ]:
        y = perf[event]
        # Rolling mean: estimate samples per smooth_ms window
        dt_ms = np.median(np.diff(perf["t_s"].values)) * 1000 if len(perf) > 2 else 1.0
        w = max(1, int(round(smooth_ms / dt_ms)))
        y_smooth = y.rolling(w, min_periods=1).mean()

        ax.plot(perf["t_s"], y_smooth, linewidth=0.6, color=color, label=label)
        shade_requests(ax, wins, color=color)
        ax.set_title(f"{label} — {event}")
        ax.set_xlabel("time (s)")
        ax.grid(True, alpha=0.2)

    axes[0].set_ylabel(f"count / {smooth_ms}ms avg")
    fig.suptitle(event, fontweight="bold", y=1.02)
    fig.tight_layout()
    plt.show()

In [ ]:
# IRQ events
irq_events = [
    "irq:irq_handler_entry",
    "irq:softirq_entry",
    "irq:softirq_raise",
    "tlb:tlb_flush",
]
for evt in irq_events:
    plot_perf_event(evt)

In [ ]:
# Software counters
sw_events = ["context-switches", "cpu-migrations", "page-faults"]
for evt in sw_events:
    plot_perf_event(evt)

In [ ]:
# Power / thermal
power_events = [e for e in n_perf.columns if "power" in e or "energy" in e or "thermal" in e or "throttle" in e]
for evt in power_events:
    plot_perf_event(evt, smooth_ms=250)

## 6 — Distribution comparison (perf events)

In [ ]:
def plot_distributions(events: list[str], bins: int = 80):
    n_evts = len(events)
    fig, axes = plt.subplots(1, n_evts, figsize=(5 * n_evts, 4))
    if n_evts == 1:
        axes = [axes]
    for ax, evt in zip(axes, events):
        if evt not in n_perf.columns or evt not in e_perf.columns:
            ax.set_title(f"{evt}\n(missing)")
            continue
        n_vals = n_perf[evt].dropna().values
        e_vals = e_perf[evt].dropna().values
        ax.hist(n_vals, bins=bins, density=True, alpha=0.5, label="neutral")
        ax.hist(e_vals, bins=bins, density=True, alpha=0.5, label="emotional")
        ax.set_title(evt)
        ax.set_xlabel("count per 1ms bucket")
        ax.legend(fontsize=8)
        ax.grid(True, alpha=0.2)
    fig.tight_layout()
    plt.show()

plot_distributions(["irq:irq_handler_entry", "irq:softirq_entry", "tlb:tlb_flush"])
plot_distributions(["context-switches", "cpu-migrations", "page-faults"])

## 7 — Descriptive statistics (perf events)

In [ ]:
def describe(series: pd.Series) -> dict:
    s = series.dropna()
    if s.empty:
        return {}
    return {
        "n": len(s),
        "mean": s.mean(),
        "std": s.std(),
        "median": s.median(),
        "p95": s.quantile(0.95),
        "p99": s.quantile(0.99),
        "max": s.max(),
        "sum": s.sum(),
    }

all_events = sorted(set(n_perf.columns) & set(e_perf.columns) - {"t_s"})
rows = []
for evt in all_events:
    nd = describe(n_perf[evt])
    ed = describe(e_perf[evt])
    rows.append({
        "event": evt,
        **{f"n_{k}": v for k, v in nd.items()},
        **{f"e_{k}": v for k, v in ed.items()},
    })

summary = pd.DataFrame(rows).set_index("event")
summary.style.format("{:.2f}", na_rep="—")

## 8 — /proc/interrupts & /proc/softirqs (200 ms resolution)

These are cumulative counters. We take first-differences to get rates.

In [ ]:
# Identify softirq columns (named types like HI, TIMER, NET_RX, etc.)
softirq_types = ["HI", "TIMER", "NET_TX", "NET_RX", "BLOCK", "IRQ_POLL",
                 "TASKLET", "SCHED", "HRTIMER", "RCU"]
softirq_cols = [c for c in softirq_types if c in n_sys.columns and c in e_sys.columns]

fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

for ax, sys_df, wins, label, color in [
    (axes[0], n_sys, n_wins, "neutral", "C0"),
    (axes[1], e_sys, e_wins, "emotional", "C1"),
]:
    for col in softirq_cols:
        rate = sys_df[col].diff() / sys_df["t_s"].diff()  # counts per second
        ax.plot(sys_df["t_s"], rate, linewidth=0.8, label=col)
    shade_requests(ax, wins, color=color)
    ax.set_title(f"{label} — /proc/softirqs rate (counts/s)")
    ax.set_ylabel("rate")
    ax.legend(fontsize=7, ncol=5, loc="upper right")
    ax.grid(True, alpha=0.2)

axes[1].set_xlabel("time (s)")
fig.tight_layout()
plt.show()

## 9 — PSI Pressure (CPU, Memory, I/O)

In [ ]:
# PSI columns have duplicated names (cpu, memory, io all have some_avg10 etc.)
# The collector writes them in order: cpu pressure, then memory, then io.
# We identify them by position in the header.

psi_cols = [c for c in n_sys.columns if c.startswith(("some_", "full_"))]
print(f"PSI columns found: {psi_cols}")

if psi_cols:
    # Just plot some_avg10 for quick overview (first occurrence = cpu)
    fig, axes = plt.subplots(1, 2, figsize=(14, 4), sharey=True)
    for ax, sys_df, wins, label, color in [
        (axes[0], n_sys, n_wins, "neutral", "C0"),
        (axes[1], e_sys, e_wins, "emotional", "C1"),
    ]:
        for col in psi_cols:
            if "avg10" in col:
                ax.plot(sys_df["t_s"], pd.to_numeric(sys_df[col], errors="coerce"),
                        linewidth=0.8, label=col)
        shade_requests(ax, wins, color=color)
        ax.set_title(f"{label} — PSI avg10")
        ax.set_xlabel("time (s)")
        ax.legend(fontsize=7)
        ax.grid(True, alpha=0.2)
    fig.tight_layout()
    plt.show()
else:
    print("No PSI columns found.")

## 10 — CPU frequency

In [ ]:
freq_cols = sorted([c for c in n_sys.columns if c.endswith("_freq_khz")])
print(f"CPU freq columns: {len(freq_cols)}")

if freq_cols:
    fig, axes = plt.subplots(1, 2, figsize=(14, 4), sharey=True)
    for ax, sys_df, wins, label, color in [
        (axes[0], n_sys, n_wins, "neutral", "C0"),
        (axes[1], e_sys, e_wins, "emotional", "C1"),
    ]:
        for col in freq_cols:
            freq_mhz = pd.to_numeric(sys_df[col], errors="coerce") / 1000
            ax.plot(sys_df["t_s"], freq_mhz, linewidth=0.4, alpha=0.5)
        # Plot mean across cores
        mean_freq = sys_df[freq_cols].apply(pd.to_numeric, errors="coerce").mean(axis=1) / 1000
        ax.plot(sys_df["t_s"], mean_freq, linewidth=1.5, color="black", label="mean")
        shade_requests(ax, wins, color=color)
        ax.set_title(f"{label} — CPU freq")
        ax.set_xlabel("time (s)")
        ax.set_ylabel("MHz")
        ax.legend(fontsize=8)
        ax.grid(True, alpha=0.2)
    fig.tight_layout()
    plt.show()
else:
    print("No CPU freq columns found (scaling_cur_freq may not be available).")

## 11 — Network & Disk I/O

In [ ]:
net_cols = sorted([c for c in n_sys.columns if c.endswith(("_rx_bytes", "_tx_bytes"))])
disk_cols = sorted([c for c in n_sys.columns if c.endswith(("_reads_completed", "_writes_completed"))])

print(f"Network columns: {net_cols}")
print(f"Disk columns: {disk_cols}")

if net_cols:
    fig, axes = plt.subplots(1, 2, figsize=(14, 4))
    for ax, sys_df, label, color in [
        (axes[0], n_sys, "neutral", "C0"),
        (axes[1], e_sys, "emotional", "C1"),
    ]:
        for col in net_cols:
            rate = pd.to_numeric(sys_df[col], errors="coerce").diff() / sys_df["t_s"].diff()
            ax.plot(sys_df["t_s"], rate / 1024, linewidth=0.8, label=col)
        ax.set_title(f"{label} — network KB/s")
        ax.set_xlabel("time (s)")
        ax.set_ylabel("KB/s")
        ax.legend(fontsize=6)
        ax.grid(True, alpha=0.2)
    fig.tight_layout()
    plt.show()

## 12 — Process-level CPU (LLM container)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 4), sharey=True)

for ax, proc, wins, label, color in [
    (axes[0], n_proc, n_wins, "neutral", "C0"),
    (axes[1], e_proc, e_wins, "emotional", "C1"),
]:
    # CPU utilization: delta(utime+stime) / delta(total)
    dt_total = proc["cpu_total_jiffies"].diff()
    dt_proc  = proc["proc_utime_jiffies"].diff() + proc["proc_stime_jiffies"].diff()
    cpu_pct = (dt_proc / dt_total * 100).clip(0, 100)
    ax.plot(proc["t_s"], cpu_pct, linewidth=0.8, color=color)
    shade_requests(ax, wins, color=color)
    ax.set_title(f"{label} — LLM container CPU %")
    ax.set_xlabel("time (s)")
    ax.set_ylabel("CPU %")
    ax.grid(True, alpha=0.2)

fig.tight_layout()
plt.show()

## 13 — Quick summary

Side-by-side descriptive stats for all perf events. Statistical tests will be added later.

In [ ]:
# Compact summary table
compact = summary[[c for c in summary.columns if "mean" in c or "std" in c or "median" in c or "sum" in c]]
compact.style.format("{:.2f}", na_rep="—")

In [ ]:
import re, json
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams["figure.dpi"] = 120


In [ ]:
# ---- CONFIG ----
# Set BASE_DIR to where you rsync'd the runs on your Mac.
# Example: BASE_DIR = Path.home() / "Desktop" / "mccviahat_runs" / "runs"
BASE_DIR = Path.home() / "Desktop" / "mccviahat_runs" / "runs"

NEUTRAL_RUN   = "2026-02-05T23-34-51_neutral"
EMOTIONAL_RUN = "2026-02-05T23-37-41_emotional"

# Alignment: "perf_start" (simplest) or "first_request"
ALIGN_MODE = "first_request"

# Smoothing windows (in milliseconds)
SMOOTH_MS_FAST = 50
SMOOTH_MS_SLOW = 250

def run_dir(name: str) -> Path:
    p = BASE_DIR / name
    if not p.exists():
        raise FileNotFoundError(f"Run dir not found: {p}")
    return p

n_dir = run_dir(NEUTRAL_RUN)
e_dir = run_dir(EMOTIONAL_RUN)

required = ["perf_stat.txt", "kernel_log.txt", "collector_meta.json", "meta.json", "responses.jsonl"]
for d in [n_dir, e_dir]:
    for f in required:
        fp = d / f
        assert fp.exists(), f"Missing {fp}"

n_dir, e_dir


In [ ]:
def load_json(path: Path) -> dict:
    return json.loads(path.read_text(encoding="utf-8"))

def load_requests(run_dir: Path) -> pd.DataFrame:
    rows = []
    for line in (run_dir / "responses.jsonl").read_text(encoding="utf-8").splitlines():
        if not line.strip():
            continue
        rows.append(json.loads(line))
    df = pd.DataFrame(rows)
    cols = ["id","title","ok","t_request_start_ns","t_request_end_ns"]
    df = df[[c for c in cols if c in df.columns]].copy()
    df = df.sort_values("t_request_start_ns").reset_index(drop=True)
    return df

def get_t0_ns(meta: dict, collector_meta: dict) -> int:
    # prefer collector start if present
    # collector_meta has t0_ns; runner meta has t0_ns too
    if "t0_ns" in collector_meta:
        return int(collector_meta["t0_ns"])
    if "t0_ns" in meta:
        return int(meta["t0_ns"])
    raise KeyError("No t0_ns found in meta/collector_meta")

n_meta = load_json(n_dir / "meta.json")
e_meta = load_json(e_dir / "meta.json")
n_cmeta = load_json(n_dir / "collector_meta.json")
e_cmeta = load_json(e_dir / "collector_meta.json")

n_req = load_requests(n_dir)
e_req = load_requests(e_dir)

n_t0 = get_t0_ns(n_meta, n_cmeta)
e_t0 = get_t0_ns(e_meta, e_cmeta)

print("Neutral requests:", len(n_req), "OK:", bool(n_req.get("ok", pd.Series([True])).all()))
print("Emotional requests:", len(e_req), "OK:", bool(e_req.get("ok", pd.Series([True])).all()))
print("Neutral perf events:", len(n_cmeta.get("perf_events", [])))
print("Emotional perf events:", len(e_cmeta.get("perf_events", [])))


In [ ]:
def parse_perf_stat(path: Path) -> pd.DataFrame:
    """Parse perf stat -I output to tidy df: t_ms, event, value."""
    lines = path.read_text(encoding="utf-8", errors="replace").splitlines()
    data = []
    # Example-ish: '  1.000123  1234  irq:softirq_entry'
    # Sometimes value is '<not counted>' or '<not supported>'; keep NaN.
    line_re = re.compile(r"^\s*(\d+(?:\.\d+)?)\s+(\S+)\s+(.+?)\s*$")
    for ln in lines:
        if not ln.strip() or ln.lstrip().startswith("#"):
            continue
        m = line_re.match(ln)
        if not m:
            continue
        t_s = float(m.group(1))
        raw = m.group(2).strip()
        evt = m.group(3).strip()
        raw_norm = raw.replace(",", "")
        val = np.nan
        if raw_norm.startswith("<"):
            val = np.nan
        else:
            try:
                val = float(raw_norm)
            except ValueError:
                val = np.nan
        data.append((t_s * 1000.0, evt, val))
    df = pd.DataFrame(data, columns=["t_ms","event","value"])
    return df

def perf_wide(df: pd.DataFrame) -> pd.DataFrame:
    w = df.pivot_table(index="t_ms", columns="event", values="value", aggfunc="first").sort_index()
    w = w.reset_index()
    w["t_s"] = w["t_ms"] / 1000.0
    return w

n_perf_tidy = parse_perf_stat(n_dir / "perf_stat.txt")
e_perf_tidy = parse_perf_stat(e_dir / "perf_stat.txt")

n_perf = perf_wide(n_perf_tidy)
e_perf = perf_wide(e_perf_tidy)

common_events = sorted(set(n_perf_tidy["event"]) & set(e_perf_tidy["event"]))
missing_n = sorted(set(e_perf_tidy["event"]) - set(n_perf_tidy["event"]))
missing_e = sorted(set(n_perf_tidy["event"]) - set(e_perf_tidy["event"]))

print("Common events:", len(common_events))
if missing_n:
    print("Events only in emotional (missing in neutral):", missing_n[:10], "..." if len(missing_n)>10 else "")
if missing_e:
    print("Events only in neutral (missing in emotional):", missing_e[:10], "..." if len(missing_e)>10 else "")
common_events[:15]


In [ ]:
def request_windows(req_df: pd.DataFrame, t0_ns: int) -> list[tuple[float,float]]:
    wins = []
    if req_df.empty:
        return wins
    for _, r in req_df.iterrows():
        s = (int(r["t_request_start_ns"]) - int(t0_ns)) / 1e9
        e = (int(r["t_request_end_ns"]) - int(t0_ns)) / 1e9
        wins.append((s, e))
    return wins

n_wins = request_windows(n_req, n_t0)
e_wins = request_windows(e_req, e_t0)

def align_to_first_request(t_s: pd.Series, wins: list[tuple[float,float]]) -> pd.Series:
    if not wins:
        return t_s
    shift = wins[0][0]
    return t_s - shift

# Align time axes if requested
if ALIGN_MODE == "first_request":
    n_perf["t_s_aligned"] = align_to_first_request(n_perf["t_s"], n_wins)
    e_perf["t_s_aligned"] = align_to_first_request(e_perf["t_s"], e_wins)
    # also shift windows for plotting
    n_wins_aligned = [(s - n_wins[0][0], e - n_wins[0][0]) for s,e in n_wins] if n_wins else []
    e_wins_aligned = [(s - e_wins[0][0], e - e_wins[0][0]) for s,e in e_wins] if e_wins else []
else:
    n_perf["t_s_aligned"] = n_perf["t_s"]
    e_perf["t_s_aligned"] = e_perf["t_s"]
    n_wins_aligned = n_wins
    e_wins_aligned = e_wins

print("ALIGN_MODE =", ALIGN_MODE)
print("Neutral first request window:", n_wins[0] if n_wins else None)
print("Emotional first request window:", e_wins[0] if e_wins else None)


## Figure 1: Run timeline overview (with request spans)

We overlay one representative tracepoint (if present) to show where activity lines up with inference windows.


In [ ]:
def shade_windows(ax, wins, alpha=0.12):
    for s,e in wins:
        ax.axvspan(s, e, alpha=alpha)

# choose a representative event if available
candidates = ["irq:softirq_entry", "irq:irq_handler_entry", "tlb:tlb_flush"]
rep = next((c for c in candidates if c in n_perf.columns and c in e_perf.columns), None)

fig, ax = plt.subplots(figsize=(10,4))
shade_windows(ax, n_wins_aligned, alpha=0.10)  # using neutral windows just for visual reference
if rep:
    ax.plot(n_perf["t_s_aligned"], n_perf[rep], label=f"neutral {rep}")
    ax.plot(e_perf["t_s_aligned"], e_perf[rep], label=f"emotional {rep}")
    ax.set_ylabel("count per 1ms bucket")
else:
    ax.text(0.02, 0.9, "No representative event found in common events.", transform=ax.transAxes)
ax.set_xlabel("time (s)")
ax.set_title("Timeline overview (request spans shaded)")
ax.grid(True, alpha=0.25)
ax.legend(loc="best")
plt.show()

rep


## Helper utilities for plotting and stats


In [ ]:
def rolling_mean_ms(series: pd.Series, t_ms: pd.Series, window_ms: int) -> pd.Series:
    # approximate rolling window by converting to number of samples using median dt
    dt = np.median(np.diff(t_ms.values)) if len(t_ms) > 2 else 1.0
    w = max(1, int(round(window_ms / dt)))
    return series.rolling(window=w, min_periods=max(1, w//4)).mean()

def describe_series(x: pd.Series) -> dict:
    x = x.dropna()
    if x.empty:
        return {"n": 0}
    return {
        "n": int(x.shape[0]),
        "mean": float(x.mean()),
        "std": float(x.std(ddof=1)) if x.shape[0] > 1 else 0.0,
        "p50": float(x.quantile(0.50)),
        "p95": float(x.quantile(0.95)),
        "p99": float(x.quantile(0.99)),
        "cv": float(x.std(ddof=1) / x.mean()) if x.mean() != 0 and x.shape[0] > 1 else np.nan,
    }

def plot_timeseries(evt: str, smooth_ms: int = SMOOTH_MS_FAST, ylabel: str = "count per 1ms bucket"):
    if evt not in n_perf.columns or evt not in e_perf.columns:
        print(f"Missing event: {evt}")
        return

    fig, ax = plt.subplots(figsize=(10,4))
    n_sm = rolling_mean_ms(n_perf[evt], n_perf["t_ms"], smooth_ms)
    e_sm = rolling_mean_ms(e_perf[evt], e_perf["t_ms"], smooth_ms)

    ax.plot(n_perf["t_s_aligned"], n_sm, label="neutral")
    ax.plot(e_perf["t_s_aligned"], e_sm, label="emotional")
    shade_windows(ax, n_wins_aligned, alpha=0.08)
    ax.set_title(f"{evt} (smoothed {smooth_ms} ms)")
    ax.set_xlabel("time (s)")
    ax.set_ylabel(ylabel)
    ax.grid(True, alpha=0.25)
    ax.legend(loc="best")
    plt.show()

def plot_distribution(evt: str, bins: int = 80):
    if evt not in n_perf.columns or evt not in e_perf.columns:
        print(f"Missing event: {evt}")
        return
    n_x = n_perf[evt].dropna().values
    e_x = e_perf[evt].dropna().values

    fig, ax = plt.subplots(figsize=(8,4))
    ax.hist(n_x, bins=bins, density=True, alpha=0.5, label="neutral")
    ax.hist(e_x, bins=bins, density=True, alpha=0.5, label="emotional")
    ax.set_title(f"{evt} distribution (per 1ms bucket)")
    ax.set_xlabel("value")
    ax.set_ylabel("density")
    ax.grid(True, alpha=0.25)
    ax.legend(loc="best")
    plt.show()

def summarize_evt(evt: str) -> pd.DataFrame:
    if evt not in n_perf.columns or evt not in e_perf.columns:
        return pd.DataFrame([{"event": evt, "error": "missing"}])
    s = {
        "event": evt,
        **{f"neutral_{k}": v for k,v in describe_series(n_perf[evt]).items()},
        **{f"emotional_{k}": v for k,v in describe_series(e_perf[evt]).items()},
    }
    return pd.DataFrame([s])


## Interrupt activity (IRQ / softIRQ)

These reflect asynchronous kernel activity. We compare smoothed time series and per-bucket distributions.


In [ ]:
irq_events = [
    "irq:irq_handler_entry",
    "irq:softirq_raise",
    "irq:softirq_entry",
    "irq:softirq_exit",
    "irq:tasklet_entry",
    "irq:tasklet_exit",
]
irq_events = [e for e in irq_events if e in common_events]
irq_events


In [ ]:
for evt in irq_events:
    plot_timeseries(evt, smooth_ms=SMOOTH_MS_FAST)
    plot_distribution(evt, bins=80)


## TLB flush activity (proxy for shootdowns)

We plot the time series and an inter-arrival-time (IAT) distribution derived from bucketed counts.


In [ ]:
TLB_EVT = "tlb:tlb_flush"
assert TLB_EVT in common_events, "tlb:tlb_flush not available in both runs"

plot_timeseries(TLB_EVT, smooth_ms=SMOOTH_MS_FAST)
plot_distribution(TLB_EVT, bins=80)


In [ ]:
def bucket_counts_to_event_times(t_s: np.ndarray, counts: np.ndarray) -> np.ndarray:
    # Expand per-bucket counts into approximate event times at bucket centers.
    # This is an approximation but sufficient for comparing IAT distributions.
    times = []
    for ts, c in zip(t_s, counts):
        if np.isnan(c) or c <= 0:
            continue
        k = int(round(c))
        times.extend([ts] * k)
    return np.array(times, dtype=float)

def plot_iat(evt: str, max_ms: int = 200, bins: int = 120):
    n_times = bucket_counts_to_event_times(n_perf["t_s_aligned"].values, n_perf[evt].values)
    e_times = bucket_counts_to_event_times(e_perf["t_s_aligned"].values, e_perf[evt].values)

    n_iat = np.diff(np.sort(n_times)) * 1000.0  # ms
    e_iat = np.diff(np.sort(e_times)) * 1000.0  # ms

    n_iat = n_iat[(n_iat >= 0) & (n_iat <= max_ms)]
    e_iat = e_iat[(e_iat >= 0) & (e_iat <= max_ms)]

    fig, ax = plt.subplots(figsize=(8,4))
    ax.hist(n_iat, bins=bins, density=True, alpha=0.5, label="neutral")
    ax.hist(e_iat, bins=bins, density=True, alpha=0.5, label="emotional")
    ax.set_title(f"Inter-arrival times for {evt} (clipped to {max_ms} ms)")
    ax.set_xlabel("IAT (ms)")
    ax.set_ylabel("density")
    ax.grid(True, alpha=0.25)
    ax.legend(loc="best")
    plt.show()

    print("Neutral IAT stats:", describe_series(pd.Series(n_iat)))
    print("Emotional IAT stats:", describe_series(pd.Series(e_iat)))

plot_iat(TLB_EVT, max_ms=200, bins=120)


## PMU jitter proxies

We compute IPC (instructions per cycle) per 1ms bucket as a simple microarchitectural stability proxy.
We compare time series (smoothed) and distributions.


In [ ]:
# Find cycles and instructions columns (names differ)
cycles_col = next((c for c in ["cycles", "cpu-cycles"] if c in n_perf.columns and c in e_perf.columns), None)
inst_col   = next((c for c in ["instructions"] if c in n_perf.columns and c in e_perf.columns), None)

print("cycles_col:", cycles_col, "inst_col:", inst_col)

assert cycles_col and inst_col, "Need cycles and instructions in perf output."

n_ipc = n_perf[inst_col] / n_perf[cycles_col]
e_ipc = e_perf[inst_col] / e_perf[cycles_col]

n_perf["ipc"] = n_ipc.replace([np.inf, -np.inf], np.nan)
e_perf["ipc"] = e_ipc.replace([np.inf, -np.inf], np.nan)

plot_timeseries("ipc", smooth_ms=SMOOTH_MS_FAST, ylabel="IPC")
plot_distribution("ipc", bins=100)

print("IPC stats neutral:", describe_series(n_perf["ipc"]))
print("IPC stats emotional:", describe_series(e_perf["ipc"]))


## Power / thermal / throttling

Energy counters are treated as counters; we take first differences and then smooth heavily.


In [ ]:
def energy_rate(series: pd.Series) -> pd.Series:
    # perf -I can report per-interval counts or cumulative depending on event; for energy it is often cumulative.
    # Taking diff is safe for comparing shape; negative diffs become NaN.
    d = series.diff()
    d[d < 0] = np.nan
    return d

energy_events = [c for c in common_events if "energy" in c]
throttle_evt = "core_power.throttle" if "core_power.throttle" in common_events else None
thermal_evt = "msr/cpu_thermal_margin/" if "msr/cpu_thermal_margin/" in common_events else None

energy_events, throttle_evt, thermal_evt


In [ ]:
# Energy plots
for evt in energy_events:
    n_perf[f"{evt}__d"] = energy_rate(n_perf[evt])
    e_perf[f"{evt}__d"] = energy_rate(e_perf[evt])

    fig, ax = plt.subplots(figsize=(10,4))
    n_sm = rolling_mean_ms(n_perf[f"{evt}__d"], n_perf["t_ms"], SMOOTH_MS_SLOW)
    e_sm = rolling_mean_ms(e_perf[f"{evt}__d"], e_perf["t_ms"], SMOOTH_MS_SLOW)
    ax.plot(n_perf["t_s_aligned"], n_sm, label="neutral")
    ax.plot(e_perf["t_s_aligned"], e_sm, label="emotional")
    shade_windows(ax, n_wins_aligned, alpha=0.08)
    ax.set_title(f"{evt} delta per 1ms bucket (smoothed {SMOOTH_MS_SLOW} ms)")
    ax.set_xlabel("time (s)")
    ax.set_ylabel("delta")
    ax.grid(True, alpha=0.25)
    ax.legend(loc="best")
    plt.show()

# Throttle
if throttle_evt:
    plot_timeseries(throttle_evt, smooth_ms=SMOOTH_MS_SLOW)
    plot_distribution(throttle_evt, bins=80)

# Thermal margin (slow)
if thermal_evt:
    plot_timeseries(thermal_evt, smooth_ms=SMOOTH_MS_SLOW, ylabel="thermal margin (units as reported)")


## Kernel log scan (MCE, hardware errors, throttling messages)

We extract lines matching key patterns. If empty, that is a valid outcome.


In [ ]:
def scan_kernel_log(path: Path, patterns: str = r"mce|machine check|hardware error|edac|thermal|throttle|watchdog") -> pd.DataFrame:
    rx = re.compile(patterns, flags=re.IGNORECASE)
    lines = path.read_text(encoding="utf-8", errors="replace").splitlines()
    hits = [ln for ln in lines if rx.search(ln)]
    return pd.DataFrame({"matches": hits})

n_hits = scan_kernel_log(n_dir / "kernel_log.txt")
e_hits = scan_kernel_log(e_dir / "kernel_log.txt")

print("Neutral kernel matches:", len(n_hits))
print("Emotional kernel matches:", len(e_hits))

display(n_hits.head(20))
display(e_hits.head(20))


## Summary table

This produces a compact comparison table for the signals you care about.


In [ ]:
summary_rows = []

# Interrupts + TLB
for evt in (irq_events + [TLB_EVT]):
    if evt in n_perf.columns and evt in e_perf.columns:
        summary_rows.append(summarize_evt(evt).iloc[0].to_dict())

# IPC
summary_rows.append(summarize_evt("ipc").iloc[0].to_dict())

# Throttle
if throttle_evt:
    summary_rows.append(summarize_evt(throttle_evt).iloc[0].to_dict())

# Energy deltas (use derived columns)
for evt in energy_events:
    col = f"{evt}__d"
    if col in n_perf.columns and col in e_perf.columns:
        # temporarily create standard names for summarizer
        n_perf[col] = n_perf[col]
        e_perf[col] = e_perf[col]
        # manual describe
        s = {"event": col}
        s.update({f"neutral_{k}": v for k,v in describe_series(n_perf[col]).items()})
        s.update({f"emotional_{k}": v for k,v in describe_series(e_perf[col]).items()})
        summary_rows.append(s)

summary = pd.DataFrame(summary_rows)
summary


In [ ]:
# Optional: export all current figures to disk (uncomment if desired)
# out_fig = Path.cwd() / "figures" / f"{NEUTRAL_RUN}__vs__{EMOTIONAL_RUN}"
# out_fig.mkdir(parents=True, exist_ok=True)
# for i, fig_num in enumerate(plt.get_fignums(), start=1):
#     fig = plt.figure(fig_num)
#     fig.savefig(out_fig / f"fig_{i:02d}.png", bbox_inches="tight")
# print("Saved to:", out_fig)
